In [2]:
import tensorflow as tf
import numpy as np
from keras.utils import load_img, img_to_array
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
from matplotlib import pyplot as plt


In [3]:
# This loads the model made in the M and Ms python script
model = tf.keras.models.load_model('m-and-m.h5')

2023-01-10 14:30:27.778956: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-10 14:30:27.779356: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1


In [4]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 49, 49, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_15 (Conv2D)          (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 23, 23, 32)       0         
 g2D)                                                            
                                                                 
 flatten_7 (Flatten)         (None, 16928)             0         
                                                                 
 dense_14 (Dense)            (None, 512)              

In [5]:
# test the model on an image
img = load_img('./test_images/IMG_1309.jpg', target_size=(100, 100))
x = img_to_array(img)
x /= 255
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
print(model.predict(images, batch_size=10))

1/1 [==============================] - 0s 193ms/step
[[0.67600816]]


2023-01-10 14:30:33.217558: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-01-10 14:30:33.303118: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [22]:
# Loop through and run sliding windows on the input image with a step size as given
imgHeight = 3024
imgWidth = 3024
stepSize = 20
img = load_img('./test_images/input_full_size.jpg', target_size=(imgWidth, imgHeight))
x = img_to_array(img)
x /= 255
x.flags["WRITEABLE"] = False

# imgWidth/stepSize = 189
# imgHeight/stepSize = 189
boxes = np.array([])
fits = np.array([])

for i in range(0, imgWidth - 100, stepSize):
    for j in range(0, imgHeight - 100, stepSize):
        topI = i + 100
        topJ = j + 100
        
        # Pull out a smaller image of size 100 x 100 out of the lager image
        croppedImg = x[i:topI,j:topJ,:]
        
        # Run the model on it
        expanded = np.expand_dims(croppedImg, axis=0)
        images = np.vstack([expanded])
        v = model.predict(images, batch_size=10, verbose=0)
        value = v[0]
        
        # I think this is legacy code but I'm afraid to remove it. I'll come back to this tomorrow
        I = 0
        J = 0
        if(i!=0):
            I = int(i/stepSize)
        if(j!=0):
            J = int(j/stepSize)
            
        # if the value is an m&m then add to the array of boxes.
        if(value[0] < 0.5):
            boxes = np.append(boxes, [j, i, topJ, topI, value[0]])
            print([i, j, topI, topJ, value[0]])


[480, 660, 580, 760, 0.45910084]
[480, 680, 580, 780, 0.32549435]
[480, 700, 580, 800, 0.2898712]
[480, 720, 580, 820, 0.36001688]
[500, 620, 600, 720, 0.35822454]
[500, 640, 600, 740, 0.18742003]
[500, 660, 600, 760, 0.08310965]
[500, 680, 600, 780, 0.044199448]
[500, 700, 600, 800, 0.034117877]
[500, 720, 600, 820, 0.039063804]
[500, 740, 600, 840, 0.06840071]
[500, 760, 600, 860, 0.16782545]
[500, 780, 600, 880, 0.45468363]
[520, 600, 620, 700, 0.3370474]
[520, 620, 620, 720, 0.15897562]
[520, 640, 620, 740, 0.053907804]
[520, 660, 620, 760, 0.020997372]
[520, 680, 620, 780, 0.012476406]
[520, 700, 620, 800, 0.0116958115]
[520, 720, 620, 820, 0.016130222]
[520, 740, 620, 840, 0.026941638]
[520, 760, 620, 860, 0.059308082]
[520, 780, 620, 880, 0.16617236]
[520, 800, 620, 900, 0.4903318]
[540, 580, 640, 680, 0.42759]
[540, 600, 640, 700, 0.22573875]
[540, 620, 640, 720, 0.067068376]
[540, 640, 640, 740, 0.017827472]
[540, 660, 640, 760, 0.0063631767]
[540, 680, 640, 780, 0.0048966026]

In [23]:
# This reshapes the boxes array to be several arrays of length 5
length = len(boxes)
newLength = round(length/5)
boxes = np.reshape(boxes, (newLength, 5))


In [24]:
# Double check the shape
print(boxes.shape)

(1118, 5)


In [34]:
# This is nonmaximum supression and intersection over union stuff that I will work on later
def NonMaximumSupression(boxes):
    minimum = smallest(boxes)
    
def intersectionOverUnion(boxes, value):
    supressed = np.array([])
    for i in boxes:
        
    
def smallest(boxes):
    minimum = [0,0,0, 0, 1]
    count = 0
    address = 0
    for i in boxes:
        if i[4] < minimum[4]:
            minimum = i
            address = count
        count += 1

    return (minimum, address)

IndentationError: expected an indented block after function definition on line 4 (381911784.py, line 6)

In [32]:
print(smallest(boxes))

(array([7.60000000e+02, 8.60000000e+02, 8.60000000e+02, 9.60000000e+02,
       1.52621593e-04]), 237)


In [33]:
print(boxes[237])

[7.60000000e+02 8.60000000e+02 8.60000000e+02 9.60000000e+02
 1.52621593e-04]


In [125]:
#@title Plot Utilities for Bounding Boxes [RUN ME]
#This is from the advanced computer vision with tensorflow course on coursera
# https://www.coursera.org/learn/advanced-computer-vision-with-tensorflow/home/info

use_normalized_coordinates = True

def draw_bounding_boxes_on_image_array(image,
                                       boxes,
                                       color=[],
                                       thickness=1,
                                       display_str_list=()):
  """Draws bounding boxes on image (numpy array).
  Args:
    image: a numpy array object.
    boxes: a 2 dimensional numpy array of [N, 4]: (ymin, xmin, ymax, xmax).
           The coordinates are in normalized format between [0, 1].
    color: color to draw bounding box. Default is red.
    thickness: line thickness. Default value is 4.
    display_str_list_list: a list of strings for each bounding box.
  Raises:
    ValueError: if boxes is not a [N, 4] array
  """
  image_pil = PIL.Image.fromarray(image)
  rgbimg = PIL.Image.new("RGBA", image_pil.size)
  rgbimg.paste(image_pil)
  draw_bounding_boxes_on_image(rgbimg, boxes, color, thickness,
                               display_str_list)
  return np.array(rgbimg)
  

def draw_bounding_boxes_on_image(image,
                                 boxes,
                                 color=[],
                                 thickness=1,
                                 display_str_list=()):
  """Draws bounding boxes on image.
  Args:
    image: a PIL.Image object.
    boxes: a 2 dimensional numpy array of [N, 4]: (ymin, xmin, ymax, xmax).
           The coordinates are in normalized format between [0, 1].
    color: color to draw bounding box. Default is red.
    thickness: line thickness. Default value is 4.
    display_str_list: a list of strings for each bounding box.
                           
  Raises:
    ValueError: if boxes is not a [N, 4] array
  """
  boxes_shape = boxes.shape
  if not boxes_shape:
    return
  if len(boxes_shape) != 2 or boxes_shape[1] != 4:
    raise ValueError('Input must be of size [N, 4]')
  for i in range(boxes_shape[0]):
    draw_bounding_box_on_image(image, boxes[i, 1], boxes[i, 0], boxes[i, 3],
                               boxes[i, 2], color, thickness)
        
def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color='red',
                               thickness=20,
                               display_str=None,
                               use_normalized_coordinates=False):
  """Adds a bounding box to an image.
  Bounding box coordinates can be specified in either absolute (pixel) or
  normalized coordinates by setting the use_normalized_coordinates argument.
  Args:
    image: a PIL.Image object.
    ymin: ymin of bounding box.
    xmin: xmin of bounding box.
    ymax: ymax of bounding box.
    xmax: xmax of bounding box.
    color: color to draw bounding box. Default is red.
    thickness: line thickness. Default value is 4.
    display_str_list: string to display in box
    use_normalized_coordinates: If True (default), treat coordinates
      ymin, xmin, ymax, xmax as relative to the image.  Otherwise treat
      coordinates as absolute.
  """
  draw = PIL.ImageDraw.Draw(image)
  im_width, im_height = image.size
  if use_normalized_coordinates:
    (left, right, top, bottom) = (xmin * imgWidth, xmax * imgWidth,
                                  ymin * imgHeight, ymax * imgHeight)
  else:
    (left, right, top, bottom) = (xmin, xmax, ymin, ymax)
  draw.line([(left, top), (left, bottom), (right, bottom),
             (right, top), (left, top)], width=thickness, fill=color)
  


In [126]:
# Draw the boxes on the image
img_to_draw = draw_bounding_boxes_on_image_array(image=np.uint8(x*255), boxes=boxes, color='green')
image_pil = PIL.Image.fromarray(np.uint8(img_to_draw))

In [127]:
#Ouput an image
image_pil.save("output.png")